# 需要先转换下模型

```bash
python .\convert.py G:\code\pretrain_model_dir\llama-7b

$env:CMAKE_ARGS = "-DLLAMA_CUBLAS=on"
pip install llama-cpp-python --no-cache-dir --verbose
```

TODO: 感觉好像没跑在 GPU 上, 需要看下文档

In [1]:
import torch
from llama_cpp import Llama

c:\tech\Anaconda3\envs\nlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
help(Llama)

Help on class Llama in module llama_cpp.llama:

class Llama(builtins.object)
 |  Llama(model_path: str, *, n_gpu_layers: int = 0, main_gpu: int = 0, tensor_split: Union[List[float], NoneType] = None, vocab_only: bool = False, use_mmap: bool = True, use_mlock: bool = False, seed: int = 4294967295, n_ctx: int = 512, n_batch: int = 512, n_threads: Union[int, NoneType] = None, n_threads_batch: Union[int, NoneType] = None, rope_freq_base: float = 0.0, rope_freq_scale: float = 0.0, mul_mat_q: bool = True, f16_kv: bool = True, logits_all: bool = False, embedding: bool = False, last_n_tokens_size: int = 64, lora_base: Union[str, NoneType] = None, lora_scale: float = 1.0, lora_path: Union[str, NoneType] = None, numa: bool = False, chat_format: str = 'llama-2', verbose: bool = True, **kwargs)
 |  
 |  High-level Python wrapper for a llama.cpp model.
 |  
 |  Methods defined here:
 |  
 |  __call__(self, prompt: str, suffix: Union[str, NoneType] = None, max_tokens: int = 128, temperature: float =

In [5]:
model_path = r"G:\code\pretrain_model_dir\llama-7b\ggml-model-f16.gguf"
llm = Llama(model_path=model_path, n_gpu_layers=-1)
# llm = Llama(model_path=model_path)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


In [6]:
output = llm("I look forward to", max_tokens=40, stop=["\n"], echo=True)
output

{'id': 'cmpl-5d383458-75f2-4833-a7e5-89e5430d4bc3',
 'object': 'text_completion',
 'created': 1697123920,
 'model': 'G:\\code\\pretrain_model_dir\\llama-7b\\ggml-model-f16.gguf',
 'choices': [{'text': "I look forward to getting back in the studio this month. The holidays are over and it's time to get back into my creative routine. I have been working on a few personal projects that will not",
   'index': 0,
   'logprobs': None,
   'finish_reason': 'length'}],
 'usage': {'prompt_tokens': 5, 'completion_tokens': 40, 'total_tokens': 45}}

In [7]:
import time
import torch
from transformers import LlamaTokenizer, LlamaTokenizerFast

model_path = r"G:\code\pretrain_model_dir\llama-7b-hf"
tokenizer = LlamaTokenizer.from_pretrained(model_path)

# 记录每次生成的时间和 token 数量
time_list = []
token_list = []

query_list = [
    "I look forward to",
    "I love beijing , because",
]
with open("./data/query.txt", "r", encoding="utf-8") as f:
    query_list = f.readlines()

for query in query_list:
    start = time.time()
    output = llm(query, max_tokens=40, stop=["\n"], echo=True)
    end = time.time()
    print(f"query: {query}, time: {end - start}")
    print(output["choices"][0]["text"])

    time_list.append(end - start)
    inputs = tokenizer(query, return_tensors="pt")
    outputs = tokenizer(output["choices"][0]["text"], return_tensors="pt")
    token_list.append(outputs.input_ids.shape[1] - inputs.input_ids.shape[1])

print(time_list)
print(token_list)
# 计算每秒生成的 token 数量
print("每秒 token 数:", sum(token_list) / sum(time_list))

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Llama.generate: prefix-match hit


query: He opened his eyes and gaspe
, time: 1.07611083984375
He opened his eyes and gaspe
He opened his eyes and gasped for air, "It would be good to have a break". The 21-year old had never played at this level before. and the team will


Llama.generate: prefix-match hit


query: She ran as fast as she coul
, time: 0.9678809642791748
She ran as fast as she coul
She ran as fast as she could until she was out of the woods. She came upon a stream and stopped to catch her breath when she heard a rustle in the trees behind her. “Oh


Llama.generate: prefix-match hit


...

query: He had a mission. A secret one
, time: 0.5708303451538086
He had a mission. A secret one
He was only supposed to be there for an hour or two, but in the end he stayed all day.


Llama.generate: prefix-match hit


query: She was a princess. But not by choice
, time: 0.47647523880004883
She was a princess. But not by choice
Maria Teresa Carafa-Spinelli died Monday at age 79


Llama.generate: prefix-match hit


query: He was a thief. A master one
, time: 0.9555888175964355
He was a thief. A master one
A master of the art, and a real professional. There are thieves who steal from their neighbors, and there are professionals who steal for the sake of it. The latter is
[1.07611083984375, 0.9678809642791748, 0.953073263168335, 0.7788467407226562, 0.9673075675964355, 0.9578988552093506, 0.9658300876617432, 0.9633538722991943, 0.8851583003997803, 0.8598403930664062, 0.9610660076141357, 0.9542670249938965, 0.45098352432250977, 0.1787266731262207, 0.9501137733459473, 0.9553947448730469, 0.953341007232666, 0.9614627361297607, 0.958432674407959, 0.9534366130828857, 0.9519116878509521, 0.4541053771972656, 0.9566679000854492, 0.9522218704223633, 0.9064345359802246, 0.6361308097839355, 0.9661595821380615, 0.5708303451538086, 0.47647523880004883, 0.9555888175964355]
[40, 40, 40, 31, 40, 40, 40, 40, 36, 35, 40, 40, 17, 5, 40, 40, 40, 40, 40, 40, 40, 17, 40, 40, 37, 25, 40, 22, 18, 40]
每秒 token 数: 40.9355892